# Big Data Modeling and Management Assigment - Homework 1

# Submission

GROUP NUMBER: **XXXXXX** - please add your group number into the file name

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|---|---|
|Benedikt Ruggaber|20240500|
|Joshua Wehr|20240501|
|Daan van Holten|20240681|
|XXXXXX|XXXXXX|

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### Problem description

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### Questions

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 
### Groups  

Groups should have 4 people maximum. Please mark which group you are here: https://shorturl.at/zE0QP 

### Submission      

The code used to produce the results and to-the-point explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 18, 2025**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and efficiency of the query (based on database schema).  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/PATH/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/PATH/Neo4Jplugins`.
- Change the code below accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW2025 -p 7474:7474 -p 7687:7687 -d -v "c:\PATH\Neo4JPlugins":/plugins -v "c:\PATH\Neo4JHWData\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

If the neo4j browser fails to load gds plugins, run the following in the Command Prompt before creating the container again:
`// Remove stopped containers //
docker container prune -f
// Remove unused images //
docker image prune -a -f
// Remove unused volumes //
docker volume prune -f
// Remove unused networks //
docker network prune -f
// Remove all unused resources in one command //
docker system prune -a -f`

In [3]:
from neo4j import GraphDatabase
from pprint import pprint

In [4]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [5]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

In [6]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

## 1 Overview of the data


In [7]:
query = """
        CALL db.schema.visualization();
    """

result = execute_read(driver, query)

pprint(result)

[<Record nodes=[<Node element_id='-12' labels=frozenset({'REVIEWS'}) properties={'name': 'REVIEWS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-11' labels=frozenset({'BEERS'}) properties={'name': 'BEERS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'COUNTRIES'}) properties={'name': 'COUNTRIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'BREWERIES'}) properties={'name': 'BREWERIES', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-13' labels=frozenset({'STYLE'}) properties={'name': 'STYLE', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'CITIES'}) properties={'name': 'CITIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-14' labels=frozenset({'USER'}) properties={'name': 'USER', 'indexes': ['name'], 'constraints': []}>] relationships=[<Relationship element_id='-9' nodes=(<Node element_id='-11' labels=frozenset({'BEERS'}) properties={'na

list all unique labels

In [8]:
query = """
MATCH (n) 
RETURN DISTINCT labels(n) AS NodeTypes;
"""

result = execute_read(driver, query)
pprint(result)  #


[<Record NodeTypes=['COUNTRIES']>,
 <Record NodeTypes=['CITIES']>,
 <Record NodeTypes=['BREWERIES']>,
 <Record NodeTypes=['BEERS']>,
 <Record NodeTypes=['REVIEWS']>,
 <Record NodeTypes=['STYLE']>,
 <Record NodeTypes=['USER']>]


list all relationsships

In [9]:
query = """
MATCH ()-[r]-() 
RETURN DISTINCT type(r) AS RelationshipTypes;
"""

result = execute_read(driver, query)
pprint(result)  # Print relationship types


[<Record RelationshipTypes='IN'>,
 <Record RelationshipTypes='BREWED'>,
 <Record RelationshipTypes='HAS_STYLE'>,
 <Record RelationshipTypes='REVIEWED'>,
 <Record RelationshipTypes='POSTED'>]


In [10]:
query = """
CALL db.stats.retrieve('GRAPH COUNTS');
"""

result = execute_read(driver, query)
pprint(result)  # Print database stats


[<Record section='GRAPH COUNTS' data={'relationships': [{'count': 5856205}, {'relationshipType': 'REVIEWED', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'startLabel': 'BEERS', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'count': 2537991, 'endLabel': 'REVIEWS'}, {'relationshipType': 'BREWED', 'count': 358873}, {'relationshipType': 'BREWED', 'startLabel': 'BREWERIES', 'count': 358873}, {'relationshipType': 'BREWED', 'count': 358873, 'endLabel': 'BEERS'}, {'relationshipType': 'IN', 'count': 62424}, {'relationshipType': 'IN', 'count': 12077, 'endLabel': 'COUNTRIES'}, {'relationshipType': 'IN', 'startLabel': 'CITIES', 'count': 12077}, {'relationshipType': 'IN', 'count': 50347, 'endLabel': 'CITIES'}, {'relationshipType': 'IN', 'startLabel': 'BREWERIES', 'count': 50347}, {'relationshipType': 'HAS_STYLE', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'startLabel': 'BEERS', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'count': 358873, 'endLabel': 'STYLE'}, {'relatio

In [11]:
query = """
CALL db.stats.retrieve('GRAPH COUNTS');
"""

result = execute_read(driver, query)
pprint(result)  # Print database stats


[<Record section='GRAPH COUNTS' data={'relationships': [{'count': 5856205}, {'relationshipType': 'REVIEWED', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'startLabel': 'BEERS', 'count': 2537991}, {'relationshipType': 'REVIEWED', 'count': 2537991, 'endLabel': 'REVIEWS'}, {'relationshipType': 'BREWED', 'count': 358873}, {'relationshipType': 'BREWED', 'startLabel': 'BREWERIES', 'count': 358873}, {'relationshipType': 'BREWED', 'count': 358873, 'endLabel': 'BEERS'}, {'relationshipType': 'IN', 'count': 62424}, {'relationshipType': 'IN', 'count': 12077, 'endLabel': 'COUNTRIES'}, {'relationshipType': 'IN', 'startLabel': 'CITIES', 'count': 12077}, {'relationshipType': 'IN', 'count': 50347, 'endLabel': 'CITIES'}, {'relationshipType': 'IN', 'startLabel': 'BREWERIES', 'count': 50347}, {'relationshipType': 'HAS_STYLE', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'startLabel': 'BEERS', 'count': 358873}, {'relationshipType': 'HAS_STYLE', 'count': 358873, 'endLabel': 'STYLE'}, {'relatio

count each node

In [12]:
query = """
MATCH (n) 
RETURN labels(n)[0] AS NodeType, count(n) AS Count
ORDER BY Count DESC;
"""

result = execute_read(driver, query)
pprint(result)  # Print node type counts


[<Record NodeType='REVIEWS' Count=2549252>,
 <Record NodeType='BEERS' Count=417746>,
 <Record NodeType='USER' Count=123935>,
 <Record NodeType='BREWERIES' Count=100694>,
 <Record NodeType='CITIES' Count=23330>,
 <Record NodeType='COUNTRIES' Count=400>,
 <Record NodeType='STYLE' Count=113>]


count relationsships

In [13]:
query = """
MATCH ()-[r]->() 
RETURN type(r) AS RelationshipType, count(r) AS Count
ORDER BY Count DESC;
"""

result = execute_read(driver, query)
pprint(result)  # Print relationship counts


[<Record RelationshipType='POSTED' Count=2538044>,
 <Record RelationshipType='REVIEWED' Count=2537991>,
 <Record RelationshipType='BREWED' Count=358873>,
 <Record RelationshipType='HAS_STYLE' Count=358873>,
 <Record RelationshipType='IN' Count=62424>]


### DATA CLEANING


city in city dosent make sense

In [14]:
query = """
MATCH (r:REVIEWS)
WHERE r.id IS NULL
RETURN COUNT(r) as null_reviews_count;
"""

result = execute_read(driver, query)
pprint(result)  # Print relationship counts


[<Record null_reviews_count=19>]


In [15]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

In [16]:
query = """
MATCH (r:REVIEWS)
WHERE r.id IS NULL
RETURN COUNT(r) as null_id_reviews
"""
result = execute_read(driver, query)
pprint(result)

[<Record null_id_reviews=19>]


In [17]:
query = """
MATCH (r:REVIEWS)
WHERE r.id IS NULL
DELETE r
RETURN COUNT(r) as deleted_reviews
"""
result = execute_write(driver, query)
pprint(result)

[<Record deleted_reviews=19>]


In [18]:
query = """
MATCH (r:REVIEWS)
WHERE r.id IS NULL
RETURN COUNT(r) as null_id_reviews
"""
result = execute_read(driver, query)
pprint(result)

[<Record null_id_reviews=19>]


In [19]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        session.execute_write(lambda tx, query: tx.run(query), query)

query = """
MATCH (c:CITIES)-[r:IN]->(c:CITIES)
DELETE r;
"""

execute_write(driver, query)
print("Self-referencing city relationships deleted successfully.")


Self-referencing city relationships deleted successfully.


In [20]:
query = """
MATCH (c:CITIES)-[r:IN]->(c:CITIES)
RETURN c, r
"""

result = execute_read(driver, query)
pprint(result)  # Should return an empty list if deletion was successful


[]


## 3. Queries

 1. How many reviews has the beer with the most reviews?
≈

In [21]:
query = """
MATCH (b:BEERS), (r:REVIEWS)
WHERE b.id = r.beer_id
WITH b, COUNT(r) AS review_count
ORDER BY review_count DESC
LIMIT 1
RETURN b.name AS beer_name, review_count;
"""

result = execute_read(driver, query)
pprint(result)  # Should return the beer with the most reviews


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.CartesianProductWarning} {category: } {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (r))} {position: line: 2, column: 1, offset: 1} for query: '\nMATCH (b:BEERS), (r:REVIEWS)\nWHERE b.id = r.beer_id\nWITH b, COUNT(r) AS review_count\nORDER BY review_count DESC\nLIMIT 1\nRETURN b.name AS beer_name, review_count;\n'


[<Record beer_name='Breakfast Stout' review_count=4833>]



    2. Which three users wrote the most reviews about beers?


In [22]:
query = """
MATCH (r:REVIEWS)
WITH r.id AS user_id
WITH user_id, COUNT(*) AS review_count
ORDER BY review_count DESC
LIMIT 3
RETURN user_id, review_count;
"""

result = execute_read(driver, query)
pprint(result)  


##TODO here is it questionable because most users have the same number of reviews and user_id is non so in the precleaning should be adapted here


[<Record user_id=None review_count=19>,
 <Record user_id='12' review_count=2>,
 <Record user_id='6' review_count=2>]


    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.


In [23]:
query = """
MATCH (b:BEERS)
WHERE 
    b.notes CONTAINS 'fruit' OR 
    b.notes CONTAINS 'complex' OR 
    b.notes CONTAINS 'nutty' OR 
    b.notes CONTAINS 'dark'
RETURN DISTINCT b.name, b.notes;
"""

result = execute_read(driver, query)
pprint(result)  # Should return beers with those descriptions


[<Record b.name='Hefeweizen' b.notes='A pale, spicy, fruity, refreshing Hefeweizen originating in Southern Germany. The fast-maturing beer is lightly hopped with hallertau and shows a unique banana-and-clove yeast character. This is a specialty for summer consumption but enjoyed year-round by all. Prost!'>,
 <Record b.name='Wheat Ale' b.notes='A hefe-weizen with exotic top notes to the aromas. Predominantly banana with floral edges. On the palate the fruit notes continue with some caramel coming through on the rich elegant finish. Retains a good mousse throughout.'>,
 <Record b.name='Philadelphia Porter' b.notes='Originally enjoyed by the working class of England, porters inevitably became a staple in American brew pubs. Our interpretation is brewed with Caramel malt for a rich, toffee-like sweetness and a touch of East Kent Goldings hops. However, it’s the robust flavor of the black and chocolate malt that take center stage in this dark, full-bodied ale.'>,
 <Record b.name='Black Cher

    4. Which top three breweries produce the largest variety of beer styles?


In [24]:
query = """
MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
WITH br.name AS brewery, COUNT(DISTINCT s.name) AS unique_styles
ORDER BY unique_styles DESC
LIMIT 3
RETURN brewery, unique_styles;
"""

result = execute_read(driver, query)
pprint(result) 


[<Record brewery='Iron Hill Brewery & Restaurant' unique_styles=94>,
 <Record brewery='Rock Bottom Restaurant & Brewery' unique_styles=93>,
 <Record brewery='Goose Island Beer Co.' unique_styles=88>]


        5. Which country produces the most beer styles?

In [25]:
query = """
MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
WITH c.name AS country, COUNT(DISTINCT s.name) AS unique_styles
ORDER BY unique_styles DESC
LIMIT 1
RETURN country, unique_styles;
"""

result = execute_read(driver, query)
pprint(result)  


[<Record country='US' unique_styles=113>]


# 4 


 Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
   
   
   
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.


## ToDo: Measure performance to communicate your improvements using PROFILE before final query.

I think she want a normal query, then the ptimized query explaining difference with PROFILE

In [26]:
query = """
MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
WHERE b.abv IS NOT NULL
WITH b, br, toFloat(b.abv) as abv_float
WHERE abv_float > 0 AND abv_float <= 67.5
RETURN b.name AS beer_name, abv_float AS abv_score, br.name AS brewery
ORDER BY abv_float DESC
LIMIT 5;
"""

result = execute_read(driver, query)
pprint(result)

[<Record beer_name='Snake Venom' abv_score=67.5 brewery='Brewmeister'>,
 <Record beer_name='series 3' abv_score=66.0 brewery='Redline Brewhouse'>,
 <Record beer_name='Armageddon' abv_score=65.0 brewery='Brewmeister'>,
 <Record beer_name='Start The Future' abv_score=60.0 brewery="Brouwerij 't Koelschip">,
 <Record beer_name='Schorschbräu Schorschbock 57%' abv_score=57.5 brewery='Schorschbräu'>]


Optimized query after using "PROFILE" to analyze:

In [28]:
query = """
MATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)
WHERE exists(b.abv) AND b.abv IS NOT NULL
WITH br, b, toFloat(b.abv) AS abv_float
WHERE abv_float > 0 AND abv_float <= 67.5
RETURN b.name AS beer_name, abv_float AS abv_score, br.name AS brewery
ORDER BY abv_float DESC
LIMIT 5;
"""

result = execute_read(driver, query)
pprint(result)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: } {title: This feature is deprecated and will be removed in future versions.} {description: The property existence syntax `... exists(variable.property)` is deprecated, please use `variable.property IS NOT NULL` instead} {position: line: 3, column: 7, offset: 49} for query: '\nMATCH (br:BREWERIES)-[:BREWED]->(b:BEERS)\nWHERE exists(b.abv) AND b.abv IS NOT NULL\nWITH br, b, toFloat(b.abv) AS abv_float\nWHERE abv_float > 0 AND abv_float <= 67.5\nRETURN b.name AS beer_name, abv_float AS abv_score, br.name AS brewery\nORDER BY abv_float DESC\nLIMIT 5;\n'


[<Record beer_name='Snake Venom' abv_score=67.5 brewery='Brewmeister'>,
 <Record beer_name='series 3' abv_score=66.0 brewery='Redline Brewhouse'>,
 <Record beer_name='Armageddon' abv_score=65.0 brewery='Brewmeister'>,
 <Record beer_name='Start The Future' abv_score=60.0 brewery="Brouwerij 't Koelschip">,
 <Record beer_name='Schorschbräu Schorschbock 57%' abv_score=57.5 brewery='Schorschbräu'>]


        2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by 

In [65]:
query = """
// Find all beer styles and calculate the average smell + feel score for each
MATCH (r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
WHERE r.smell IS NOT NULL 
  AND r.feel IS NOT NULL
  AND s.name IS NOT NULL
  AND s.name <> "nan"
  AND s.name <> ""

// Calculate the average smell + feel score for each style
WITH s.name AS beer_style, AVG(toFloat(r.smell) + toFloat(r.feel)) AS avg_score
WHERE avg_score IS NOT NULL

// Return the top 5 styles with highest average score
RETURN beer_style, avg_score
ORDER BY avg_score DESC
LIMIT 5;"""

result = execute_read(driver, query)
pprint(result)

[<Record beer_style='New England IPA' avg_score=8.792995379793723>,
 <Record beer_style='American Imperial Stout' avg_score=8.49245855991279>,
 <Record beer_style='Belgian Gueuze' avg_score=8.421866737176126>,
 <Record beer_style='American Imperial Porter' avg_score=8.326940877010319>,
 <Record beer_style='Russian Imperial Stout' avg_score=8.29099522620976>]


### 5 

5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?

1:

In [66]:
query = """
// Step 1: Find top 5 beer styles for each country
MATCH (c:COUNTRIES)<-[:IN]-(:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
WHERE c.name IS NOT NULL AND s.name IS NOT NULL AND s.name <> "nan" AND s.name <> ""
WITH c.name AS country, s.name AS style, COUNT(b) AS beer_count
ORDER BY country, beer_count DESC
WITH country, COLLECT(style)[0..5] AS top_styles
WHERE SIZE(top_styles) = 5

// Step 2: Compare all country pairs
WITH COLLECT({country: country, styles: top_styles}) AS country_data
UNWIND country_data AS cd1
UNWIND country_data AS cd2
WITH cd1, cd2
WHERE cd1.country < cd2.country  // Now the WHERE clause is after a WITH

// Step 3: Calculate similarity (number of common styles)
WITH cd1.country AS country1, cd2.country AS country2,
     [s IN cd1.styles WHERE s IN cd2.styles] AS common_styles
WITH country1, country2, common_styles, SIZE(common_styles) AS similarity

// Step 4: Return the most similar pair
RETURN country1, country2, common_styles, similarity
ORDER BY similarity DESC
LIMIT 1;
"""

result = execute_read(driver, query)
pprint(result)

[<Record country1='AU' country2='NZ' common_styles=['American IPA', 'American Pale Ale (APA)', 'American Imperial IPA', 'Belgian Saison', 'American Imperial Stout'] similarity=5>]


2:

In [67]:
query = """

MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)-[:POSTED]->(u:USER)
WITH b.name AS beer_name, b.id AS beer_id, COUNT(DISTINCT u) AS user_count
ORDER BY user_count DESC
LIMIT 1
RETURN beer_name, beer_id, user_count;

"""

result = execute_read(driver, query)
pprint(result)

[<Record beer_name='Breakfast Stout' beer_id='11757' user_count=4828>]


3: skipped it because to hard the way i did it atleast

4:     4. Which user is the most influential when it comes to reviews of distinct beers by style?

In [68]:
query = """

MATCH (u:USER)<-[:POSTED]-(r:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:HAS_STYLE]->(s:STYLE)
WHERE s.name IS NOT NULL AND s.name <> "nan" AND s.name <> ""
WITH u.name AS user_name, COUNT(DISTINCT s.name) AS distinct_styles_reviewed
ORDER BY distinct_styles_reviewed DESC
LIMIT 1
RETURN user_name, distinct_styles_reviewed;
"""

result = execute_read(driver, query)
pprint(result)



[<Record user_name='kojevergas' distinct_styles_reviewed=110>]
